In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.7 MB/s 
     |████████████████████████████████| 120 kB 59.1 MB/s 


In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
import pandas as pd
import numpy as np

import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel
from keras.models import Sequential, Model

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [8]:
data = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/allData-19-08-27.ta'
df = pd.read_csv(data, header=None, names=['news'])
df.head()

,news
0,அமைச்சின் பணிகளை முன்னெடுப்பதற்கு கௌர அமைச்சர்...
1,இவ்வமைச்சு இல 40 புத்கமுவ வீதி இராஜகிரிய எனு...
2,தேசிய மொழிகள் மற்றும் சமூக ஒருமைப்பாட்டு அமைச்...
3,தேசிய மொழிகள் சமூக ஒருமைப்பாடு தொடர்பான கொள்...
4,குறித்த தேசிய நோக்கங்களை எய்தும் பொருட்டு தேச...


In [ ]:
df['news'].str.len().max()

7422

# BERT Tokenizer

In [10]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
bert_model = transformers.TFBertModel.from_pretrained('bert-base-multilingual-uncased')

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def bert_encode(data, maximum_length) :
  input_ids = []
  attention_masks = []
  
  for i in range(len(data)):
    encoded = tokenizer.encode_plus(      
      data[i],
      add_special_tokens = True,
      max_length = maximum_length,
      padding=True,
      truncation = True,
      return_attention_mask = True      
    )
  
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
  
  return np.array(input_ids), np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = bert_encode(df['news'], 512)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')

bert_output = bert_model([input_ids, input_masks_ids])[0]
bert_output.shape
bert_output[:, 0, :]

model = Model(inputs=[input_ids, input_masks_ids], outputs = [bert_output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  167356416   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
#Use the tokenizer and model  from the Transformers and determine the output features from the last hidden layer.
def get_embeddings(model_name, tokenizer, name, inp):
  tokenizer = tokenizer.from_pretrained(name)
  model = model_name.from_pretrained(name)

  input_ids = tf.constant(tokenizer.encode(inp))[None, :]  # Batch size 1
  outputs = model(input_ids)

  last_hidden_states = outputs[0]
  cls_token = last_hidden_states[0]

  return cls_token

In [ ]:
z = df['news'].tolist()
z[0]

'அமைச்சின் பணிகளை முன்னெடுப்பதற்கு கௌர அமைச்சர் வாசுதேவ நாணாயக்கார அவர்களுக்கு தத்துவம் வழங்கப்பட்டதுடன் திருமதி எம்  எஸ்  விக்கிரமசிங்க அமைச்சின் செயலாளராக செயலாற்றுகிறார் '

In [ ]:
cls_token = get_embeddings(TFBertModel, BertTokenizer, 'bert-base-multilingual-uncased', z[0])
cls_token

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


<tf.Tensor: shape=(71, 768), dtype=float32, numpy=
array([[-0.10861358, -0.06730223,  0.07388302, ...,  0.04361111,
        -0.04630373, -0.06499264],
       [ 0.05621428,  0.11254698, -0.0088466 , ..., -0.03384805,
         0.10702873, -0.42685288],
       [-0.35092622, -0.83444476,  0.6366284 , ...,  0.19836532,
        -0.6008336 , -0.12423165],
       ...,
       [-0.4456465 , -0.3897203 ,  0.796821  , ...,  0.70930153,
        -0.54098004, -1.0375787 ],
       [ 0.08041987,  0.01899316,  0.2989576 , ...,  1.0375788 ,
        -0.54234517, -0.49848205],
       [-0.30701053,  0.23436832,  0.26524258, ...,  0.576362  ,
         0.04702273, -0.7759421 ]], dtype=float32)>

Run it for all

In [11]:
tokenizer = tokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
def get_vectors(inp):
  input_ids = tf.constant(tokenizer.encode(inp))[None, :]
  outputs = model(input_ids)

  last_hidden_states = outputs[0]
  cls_token = last_hidden_states[0]

  return cls_token

In [13]:
embeddings = []

for index, row in tqdm(df.iterrows()):
  vec = get_vectors(row['news'])
  embeddings.append(vec)

  break

0it [00:01, ?it/s]


In [ ]:
# Convert the list of vectors into a 2D array.
vecs = np.stack(embeddings)
vecs.shape

In [ ]:
np.save('/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/vectors/embeddings.npy', vecs)

## Method 2

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 120 kB 41.1 MB/s 


In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel
from keras.models import Sequential, Model

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [5]:
data = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/allData-19-08-27.ta'
df = pd.read_csv(data, header=None, names=['news'])
df.head(), len(df)

(                                                news
 0  அமைச்சின் பணிகளை முன்னெடுப்பதற்கு கௌர அமைச்சர்...
 1  இவ்வமைச்சு இல 40  புத்கமுவ வீதி  இராஜகிரிய எனு...
 2  தேசிய மொழிகள் மற்றும் சமூக ஒருமைப்பாட்டு அமைச்...
 3   தேசிய மொழிகள்  சமூக ஒருமைப்பாடு தொடர்பான கொள்...
 4   குறித்த தேசிய நோக்கங்களை எய்தும் பொருட்டு தேச..., 1727888)

In [ ]:
# tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
# bert_model = transformers.TFBertModel.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
# tokenizer = tokenizer.from_pretrained('bert-base-multilingual-uncased')
# model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

In [6]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
def get_vectors(inp):
  input_ids = tf.constant(tokenizer.encode(inp,
                                           add_special_tokens = True,
                                           max_length = 512, 
                                           truncation = True, 
                                           padding = "max_length"))[None, :]
  outputs = model(input_ids)

  last_hidden_states = outputs[0]
  cls_token = last_hidden_states[0]

  return cls_token

In [ ]:
df['vec'] = df['news'].progress_apply(get_vectors)

  0%|          | 0/1727888 [00:00<?, ?it/s]

In [ ]:
# # using for-loop
# embeddings = []

# for index, row in tqdm(df.iterrows()):
#   vec = get_vectors(row['news'])
#   embeddings.append(vec)

In [ ]:
# Convert the list of vectors into a 2D array.
vecs = np.stack(embeddings)
vecs.shape

In [ ]:
np.save('/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/vectors/embeddings.npy', vecs)